Back to the main [Index](../index.ipynb)

### Model Loading

To load models use any the following functions. Each function takes a model with the corresponding format and converts it to a [RoadRunner](http://sys-bio.github.io/roadrunner/python_docs/index.html) simulator instance.

* `te.loadAntimony` (`te.loada`): Load an Antimony model.
* `te.loadSBML`: Load an SBML model.
* `te.loadCellML`: Load a CellML model (this passes the model through Antimony and converts it to SBML, may be lossy).

In [1]:
from __future__ import print_function
import tellurium as te
te.setDefaultPlottingEngine('matplotlib')
%matplotlib inline

model = """
model test
    compartment C1;
    C1 = 1.0;
    species S1, S2;
    
    S1 = 10.0;
    S2 = 0.0;
    S1 in C1; S2 in C1;
    J1: S1 -> S2; k1*S1;
    
    k1 = 1.0;
end
"""
# load models
r = te.loada(model)

### Running Simulations

Simulating a model in roadrunner is as simple as calling the `simulate` function on the RoadRunner instance `r`. The simulate acceps three positional arguments: start time, end time, and number of points. The simulate function also accepts the keyword arguments `selections`, which is a list of variables to include in the output, and `steps`, which is the number of integration time steps and can be specified instead of number of points.

In [2]:
# simulate from 0 to 50 with 100 steps
r.simulate(0, 50, 100)
# plot the simulation
r.plot()

### Integrator and Integrator Settings
To set the integrator use `r.setIntegrator(<integrator-name>)` or `r.integrator = <integrator-name>`. RoadRunner supports `'cvode'`, `'gillespie'`, and `'rk4'` for the integrator name. CVODE uses adaptive stepping internally, regardless of whether the output is gridded or not. The size of these internal steps is controlled by the tolerances, both absolute and relative.

To set integrator settings use `r.integrator.<setting-name> = <value>` or `r.integrator.setValue(<setting-name>, <value>)`. Here are some important settings for the `cvode` integrator:

* `variable_step_size`: Adaptive step-size integration (`True`/`False`).
* `stiff`: Stiff solver for CVODE only (`True`/`False`). Enabled by default.
* `absolute_tolerance`: Absolute numerical tolerance for integrator internal stepping.
* `relative_tolerance`: Relative numerical tolerance for integrator internal stepping.

Settings for the `gillespie` integrator:

* `seed`: The RNG seed for the Gillespie method. You can set this before running a simulation, or leave it alone for a different seed each time. Simulations initialized with the same seed will have the same results.

In [3]:
# what is the current integrator?
print('The current integrator is:')
print(r.integrator)

# enable variable stepping
r.integrator.variable_step_size = True
# adjust the tolerances (can set directly or via setValue)
r.integrator.absolute_tolerance = 1e-3 # set directly via property
r.integrator.setValue('relative_tolerance', 1e-1) # set via a call to setValue

# run a simulation, stop after reaching or passing time 10
r.reset()
results = r.simulate(0, 10)
r.plot()

# print the time values from the simulation
print('Time values:')
print(results[:,0])

In [4]:
# set integrator to Gillespie solver
r.setIntegrator('gillespie')
# identical ways to set integrator
r.setIntegrator('rk4')
r.integrator = 'rk4'
# set back to cvode (the default)
r.setIntegrator('cvode')

# set integrator settings
r.integrator.setValue('variable_step_size', False)
r.integrator.setValue('stiff', True)

# print integrator settings
print(r.integrator)

### Simulation options

The `RoadRunner.simulate` method is responsible for running simulations using the current integrator. It accepts the following arguments:

* `start`: Start time.
* `end`: End time.
* `points`: Number of points in solution (exclusive with steps, do not pass both). If the output is gridded, the points will be evenly spaced in time. If not, the simulation will stop when it reaches the `end` time or the number of points, whichever happens first.
* `steps`: Number of steps in solution (exclusive with points, do not pass both).



In [5]:
# simulate from 0 to 6 with 6 points in the result
r.reset()
# pass args explicitly via keywords
res1 = r.simulate(start=0, end=10, points=6)
print(res1)
r.reset()
# use positional args to pass start, end, num. points
res2 = r.simulate(0, 10, 6)
print(res2)

### Selections

The selections list can be used to set which state variables will appear in the output array. By default, it includes all SBML species and the `time` variable. Selections can be either given as argument to `r.simulate`.

In [6]:
print('Floating species in model:')
print(r.getFloatingSpeciesIds())
# provide selections to simulate
print(r.simulate(0,10,6, selections=r.getFloatingSpeciesIds()))
r.resetAll()
# try different selections
print(r.simulate(0,10,6, selections=['time','J1']))

### Reset model variables
To reset the model's state variables use the `r.reset()` and `r.reset(SelectionRecord.*)` functions. If you have made modifications to parameter values, use the `r.resetAll()` function to reset parameters to their initial values when the model was loaded.

In [7]:
# show the current values
for s in ['S1', 'S2']:
    print('r.{} == {}'.format(s, r[s]))
# reset initial concentrations
r.reset()
print('reset')
# S1 and S2 have now again the initial values
for s in ['S1', 'S2']:
    print('r.{} == {}'.format(s, r[s]))
# change a parameter value
print('r.k1 before = {}'.format(r.k1))
r.k1 = 0.1
print('r.k1 after = {}'.format(r.k1))
# reset parameters
r.resetAll()
print('r.k1 after resetAll = {}'.format(r.k1))